In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
!pip install simpletransformers


     |████████████████████████████████| 194kB 2.9MB/s 
     |████████████████████████████████| 204kB 7.6MB/s 
     |████████████████████████████████| 3.8MB 9.0MB/s 
     |████████████████████████████████| 675kB 29.0MB/s 
     |████████████████████████████████| 890kB 38.0MB/s 
     |████████████████████████████████| 1.1MB 40.8MB/s 
  Created wheel for seqeval: filename=seqeval-0.0.12-cp36-none-any.whl size=7424 sha256=5da63b6dc7ea1c0e0953a045af8cfc8c20f40eb44856cd7969835bcee028a7b2
  Stored in directory: /root/.cache/pip/wheels/4f/32/0a/df3b340a82583566975377d65e724895b3fad101a3fb729f68
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=f568b75cdd802648c0ade1bdd25b49a44e125bbae9c45c5c113eff8eddf459ba
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built seqeval sacremoses


In [4]:
cd drive/Shared\ drives/Language\ Technology\ Project/Final\ Project/Code/LtP/

/content/drive/Shared drives/Language Technology Project/Final Project/Code/LtP


In [5]:
import torch
torch.cuda.is_available()

True

In [6]:
%%writefile setup.sh

# git clone https://github.com/NVIDIA/apex
pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex

Overwriting setup.sh


In [7]:
!sh setup.sh

/usr/local/lib/python3.6/dist-packages/pip/_internal/commands/install.py:283: UserWarning: Disabling all use of wheels due to the use of --build-options / --global-options / --install-options.
  cmdoptions.check_install_build_global(options)
Created temporary directory: /tmp/pip-ephem-wheel-cache-fhsx6zaq
Created temporary directory: /tmp/pip-req-tracker-sh4mjzxb
Created requirements tracker '/tmp/pip-req-tracker-sh4mjzxb'
Created temporary directory: /tmp/pip-install-27y98t3l
Processing ./apex
  Created temporary directory: /tmp/pip-req-build-cjg5t_cj
  Added file:///content/drive/Shared%20drives/Language%20Technology%20Project/Final%20Project/Code/LtP/apex to build tracker '/tmp/pip-req-tracker-sh4mjzxb'
    Running setup.py (path:/tmp/pip-req-build-cjg5t_cj/setup.py) egg_info for package from file:///content/drive/Shared%20drives/Language%20Technology%20Project/Final%20Project/Code/LtP/apex
    Running command python setup.py egg_info


    torch.__version__  = 1.5.0+cu101


    run

In [0]:

import pandas as pd
import argparse
import numpy as np
from sklearn.model_selection import train_test_split
from simpletransformers.classification import ClassificationModel
from transformers import BertModel, BertTokenizer
from os import path
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
import json 

In [0]:
def get_test_dataframe():
  """Returns a pandas dataframe with the correct label"""
  test_en = [[line.rstrip(), 0] for line in open('data/test_src_en.txt', encoding='utf-8').readlines()]
  test_deepl = [[line.rstrip(), 1] for line in open('data/test_deepL_en.txt', encoding='utf-8').readlines()]
  return pd.DataFrame(test_en + test_deepl)


In [10]:
"""Obtain predictions"""
test = get_test_dataframe()
model_BERT = ClassificationModel('bert', 'bert_outputs_20_epochs/')
predictions, raw_outputs = model_BERT.predict(test[0].to_list())

In [11]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertModel.from_pretrained('bert_outputs_20_epochs/', output_attentions=True)
"""Creates a set of dictionaries with text, labels, predictions and attention
scores to use for the visualisation tool."""
output_set = list()
count = 0
for index, row in test.iterrows():
  inputs = tokenizer.encode_plus(row[0], return_tensors='pt', add_special_tokens=True) #encode words for compatability with model
  input_ids = inputs['input_ids']
  attention = model(input_ids)[-1] #obtain attention scores
  input_id_list = input_ids[0].tolist() # Batch index 0
  tokens = tokenizer.convert_ids_to_tokens(input_id_list) #convert ids back to tokens
  all_layers = torch.sum(attention[-1], dim=1).squeeze() / 12 #sum attention scores per word and divide by amount of layers
  scores_per_word = (torch.sum(all_layers, dim=0) /len(tokens)).tolist() #Obtain flat list of scores from len(tokens) * len(tokens)
  output_dict = {'text': [], 'attention': []}
  output_dict['label'] = int(row[1])
  output_dict['prediction'] = int(predictions[index])
  count += 1
  output_dict['id'] = str(count) #neat-vision requires a unique id per sentence
  output_dict['text'] = tokens
  output_dict['attention'] = scores_per_word
  output_set.append(output_dict)
with open('text.json', 'w') as outfile:
  json.dump(output_set, outfile)

#put in json output here: https://cbaziotis.github.io/neat-vision/ 



In [0]:
label_dict = { "0": {"name": "Source translation", "desc": "Source translation"}, 
              "1": {"name":"DeepL translation", "desc":"DeepL translation"}}
with open('labels.json', 'w') as outfile:
  json.dump(label_dict, outfile) 